# Calculation of low-temperature viscosity of decane with reverse non-equilibrium MD

In [ ]:
model_version = "v7.0.0"   # Placeholder
calc_mode = "crystal_u0_plus_d3"    # Placeholder

## 1. Initial structures

In [ ]:
import numpy as np
from pathlib import Path
from IPython.display import clear_output
from ase import Atoms
from ase.io import read, write

from pfcc_extras.structure.ase_rdkit_converter import smiles_to_atoms
from pfcc_extras.liquidgenerator.liquid_generator import LiquidGenerator

# SMILES string for n-decane
smiles_ndecane = "CCCCCCCCCC"

def make_liquid_ndecane(n_mol, density):
    """
    Generate a liquid configuration for n-decane with a specified number of molecules and density.
    
    Args:
      n_mol (int): Number of n-decane molecules in the configuration.
      density (float): Target density (g/cm^3) to pack the molecules.
      
    Returns:
      atoms (Atoms): ASE Atoms object representing the packed n-decane liquid.
    """
    # Create a list of n-decane molecules with varied conformations by using different random seeds.
    mol = smiles_to_atoms(smiles_ndecane)
    composition = [mol] * n_mol
    
    # Use LiquidGenerator to pack the molecules in a cubic simulation box.
    liquid_generator = LiquidGenerator(engine="torch", composition=composition, density=density, cubic=True)
    atoms = liquid_generator.run(epochs=100)
    clear_output()  # Clear intermediate outputs if running in interactive environments.
    return atoms

def make_random_ndecane(size="small", density=0.73):
    """
    Generate a random n-decane liquid structure.

    Args:
      size (str): "small" or "large" structure. 
                  "small" corresponds to ~5-10 molecules, while "large" corresponds to ~10-20 molecules.
      density (float): Target density (g/cm^3) for packing the molecules.
    
    Returns:
      atoms (Atoms): Generated ASE Atoms object.
    """
    assert size in ["small", "large"], "Size must be either 'small' or 'large'."
    if size == "small":
        n_mol = np.random.randint(5, 11)  # roughly 5 to 10 molecules
    else:
        n_mol = np.random.randint(10, 21)  # roughly 10 to 20 molecules
    
    atoms = make_liquid_ndecane(n_mol, density)
    return atoms

# Create directory to save generated structures
structure_dir = Path("structures")
structure_dir.mkdir(parents=True, exist_ok=True)

# List to collect initial structures
initial_structures = []

# Generate initial configurations sampling different densities.
# Variations in density can help cover a range of possible molecular arrangements.
for density in [0.65, 0.73, 0.80]:
    for size in ["small", "large"]:
        for i in range(3):  # Create 3 configurations for each combination
            filename = f"ndecane_{size}_dens{density:.2f}_{i}.xyz"
            filepath = structure_dir / filename
            if not filepath.is_file():
                atoms = make_random_ndecane(size=size, density=density)
                atoms.write(filepath)
            else:
                atoms = read(filepath)
            initial_structures.append(atoms)

## 2. Initial dataset

In [ ]:
from pathlib import Path
from concurrent.futures import as_completed, ThreadPoolExecutor
import numpy as np
from h5py import File
from tqdm.auto import tqdm

from pfp_api_client import Estimator, ASECalculator
from light_pfp_data.utils.dataset import H5DatasetWriter
from light_pfp_data.sample.crystal import sample_md, sample_rattle

# Create a folder for the initial dataset if it does not exist.
init_dataset_dir = Path("init_dataset")
init_dataset_dir.mkdir(parents=True, exist_ok=True)

# Define the initial dataset file path.
initial_dataset = init_dataset_dir / "init.h5"

# Check for existence of dataset file; if exists, load it. Otherwise, sample to generate new data.
if initial_dataset.exists():
    print(f"Dataset file {initial_dataset} already exists. Skipping dataset generation.")
    dataset = H5DatasetWriter(File(initial_dataset))
else:
    print(f"Dataset file {initial_dataset} does not exist. Starting initial dataset sampling.")
    dataset = H5DatasetWriter(initial_dataset)
    estimator = Estimator(model_version=model_version, calc_mode=calc_mode)
    calc = ASECalculator(estimator)
    
    # For multithreaded execution.
    futures = []
    pbar = tqdm(desc="Total progress", total=0, leave=True)
    
    # "initial_structures" is assumed to be available from the previous initial structure generation step.
    # It contains several n-decane initial configurations.
    with ThreadPoolExecutor(max_workers=16) as executor:
        for atoms in initial_structures:
            # MD sampling with NVT ensemble at high temperatures for robust coverage.
            # Temperatures: 500K, 1000K, 1500K
            # For each temperature, run 5000 steps and sample one structure every 100 steps.
            futures += sample_md(
                input_structure=atoms,
                calculator=calc,
                dataset=dataset,
                supercell=(1, 1, 1),
                sampling_temp=[500.0, 1000.0, 1500.0],
                sampling_steps=[2000, 2000, 2000],
                sampling_interval=[100, 100, 100],
                ensemble="nvt",
                executor=executor,
                pbar=pbar
            )
            # MD sampling with NPT ensemble at target viscosity conditions.
            # Temperatures: 300K, 400K, 500K; Pressure: 1.0 bar.
            futures += sample_md(
                input_structure=atoms,
                calculator=calc,
                dataset=dataset,
                supercell=(1, 1, 1),
                sampling_temp=[300.0, 400.0, 500.0, 600.0],
                sampling_pressure=[1.0, 1.0, 1.0, 1.0],
                sampling_steps=[2000, 2000, 2000, 2000],
                sampling_interval=[100, 100, 100, 100],
                ensemble="npt",
                executor=executor,
                pbar=pbar
            )
            # Rattle sampling: random atomic displacements with a small standard deviation.
            futures += sample_rattle(
                input_structure=atoms,
                calculator=calc,
                dataset=dataset,
                stdev=0.1,
                n_sample=10,
                supercell=(1, 1, 1)
            )
            # Rattle sampling with a larger displacement.
            futures += sample_rattle(
                input_structure=atoms,
                calculator=calc,
                dataset=dataset,
                stdev=0.15,
                n_sample=10,
                supercell=(1, 1, 1)
            )
    
    # Wait for all sampling tasks to complete.
    for future in as_completed(futures):
        _ = future.result()

# Close the dataset file.
dataset.h5.close()

## 3. Active learning

In [ ]:
import logging
from light_pfp_autogen.active_learning import ActiveLearning
from light_pfp_autogen.config import ActiveLearningConfig, TrainConfig, SampleConfig, CommonConfig, MTPConfig

# Configure logging to show active learning process details.
logging.basicConfig(level=logging.INFO)

# Use previously defined model_version and calc_mode from earlier steps.
# For n-decane (an organic molecule), we use the ORGANIC_SMALL_NN pretrained model.
active_learning_config = ActiveLearningConfig(
    task_name="ndecane-viscosity",
    pfp_model_version=model_version,
    pfp_calc_mode=calc_mode,
    init_dataset=["init_dataset/init.h5"],
    train_config=TrainConfig(
        common_config=CommonConfig(max_forces=50.0),
        mtp_config=MTPConfig(pretrained_model="ORGANIC_SMALL_NN")
    ),
    sample_config=SampleConfig(
        dE_min_coef=3.0,
        dE_max_coef=20.0,
        dF_min_coef=10.0,
        dF_max=50.0,
        dS_min_coef=3.0,
        dS_max_coef=20.0,
    )
)

# Initialize the active learning task with the specified configuration.
active_learning = ActiveLearning(active_learning_config)
active_learning.initialize()

In [ ]:
import numpy as np
from ase import units
from ase.md.langevin import Langevin
from ase.md.nvtberendsen import NVTBerendsen
from ase.md.npt import NPT
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from IPython.display import clear_output
from light_pfp_autogen.context import DataCollectionContext
from rnemd_extensions import RNEMDExtension


def active_learning_protocol(atoms, temperature, steps_npt, steps_rnemd, n_swap, swap_interval):
    # Initialize atomic velocities according to Maxwell-Boltzmann distribution.
    MaxwellBoltzmannDistribution(atoms, temperature_K=temperature)
    
    # Run a short NVT MD simulation (e.g., using Langevin dynamics) to equilibrate the structure.
    md = Langevin(atoms, 1.0 * units.fs, temperature_K=temperature, friction=0.1)
    with DataCollectionContext(md=md, interval=100, max_samples=20):
        md.run(steps=5000)
    
    # Then run a longer NPT MD simulation to generate diverse training structures.
    md = NPT(
        atoms,
        1.0 * units.fs,
        temperature_K=temperature,
        externalstress=1.0 * units.bar,
        mask=np.eye(3),
        ttime=20.0 * units.fs,
        pfactor=2e6 * units.GPa * (units.fs**2)
    )
    with DataCollectionContext(md=md, interval=100, max_samples=steps_npt // 100 // 2):
        md.run(steps=steps_npt)

    md = NVTBerendsen(
        atoms,
        1.0 * units.fs,
        temperature_K=temperature,
        taut=500.0 * units.fs
    )
    rnemd = RNEMDExtension(
        md,
        rnemd_type="viscosity",
        n_slab=10,
        n_swap=n_swap,
        swap_interval=swap_interval,
    )
    md.attach(rnemd)
    with DataCollectionContext(md=md, interval=100, max_samples=steps_rnemd // 100 // 2):
        md.run(steps=steps_rnemd)
        
    clear_output()


for i in range(active_learning.iter, 5):
    print(f"Active learning iteration: {i} (small structure)")
    for _ in range(10):
        temperature = np.random.uniform(300, 600)
        atoms = make_random_ndecane("small", density=0.73)
        n_swap = 1
        swap_interval = np.random.choice([10, 20, 50, 100, 200, 500, 1000])
        print(f"Running MD for n-decane: size = {size}, temperature = {temperature:.1f} K")
        active_learning_protocol(atoms, temperature, steps_npt=10000, steps_rnemd=20000, n_swap=n_swap, swap_interval=swap_interval)
    active_learning.update()

for i in range(active_learning.iter, 10):
    print(f"Active learning iteration: {i} (large structure)")
    for _ in range(5):
        temperature = np.random.uniform(300, 600)
        atoms = make_random_ndecane("small", density=0.73)
        n_swap = 1
        swap_interval = np.random.choice([10, 20, 50, 100, 200, 500, 1000])
        active_learning_protocol(atoms, temperature, steps_npt=20000, steps_rnemd=30000, n_swap=n_swap, swap_interval=swap_interval)
    active_learning.update()

active_learning.print_md_statistics()

## 4. Post training

In [ ]:
from light_pfp_autogen.utils import submit_training_job, check_training_job_status, estimate_epoch

epoch = estimate_epoch(active_learning.datasets_list, 2)
train_config_dict = {
    "common_config": {
        "total_epoch": epoch,
        "max_forces": 50.0
    },
    "mtp_config": {
        "pretrained_model": "ORGANIC_SMALL_NN"
    },
}

training_config = TrainConfig.from_dict(
    train_config_dict
)

model_id = submit_training_job(
    training_config,
    active_learning.datasets_list,
    "ndecane-viscosity-final",
)

status = check_training_job_status(model_id)
print(f"Training job {model_id} status: {status}")

## 5. PFP validation MD

* To compare the reliablity of LightPFP model in the viscosity calculation, the reverse non-equilibrium MD (rNEMD) simulation is performed at 400K, 450K and 500K. The relatively high temperature is selected because the convergenced result is easier to achieve at higher temperature.
* One important parameter in rNEMD is the momenta exchange frequency. Usually, the lower frqeuency (higher interval) means more closer to the real experimental situation, but, requires longer MD to get converged results. Here, exchange interval is set to 100 and 500.
* MD workflow
    * Start from "asset/test.xyz" initial structure
    * Run NVT MD for 5 ps
    * Run NPT MD for 20 ps
    * Run NVT MD + rNEMD for 50 ps
* Conditions for rNEMD
    * Number of slabs: 20
    * Exchange interval: 100 / 500
    * Calculation of velocity profile: every 10 MD step

In [ ]:
import numpy as np
from ase import units
from ase.io import Trajectory
from ase.md.langevin import Langevin
from ase.md.nvtberendsen import NVTBerendsen
from ase.md.npt import NPT
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from IPython.display import clear_output
from light_pfp_autogen.context import DataCollectionContext
from rnemd_extensions import RNEMDExtension, VelocityLogExtension


def md_script(atoms, temperature, steps_npt, steps_rnemd, n_swap, swap_interval, log_interval, logfile, profile, trajfile):
    n_slab = 20
    traj = Trajectory(trajfile, "w", atoms=atoms)
    MaxwellBoltzmannDistribution(atoms, temperature_K=temperature)
    
    # Run a short NVT MD simulation (e.g., using Langevin dynamics) to equilibrate the structure.
    md = Langevin(atoms, 1.0 * units.fs, temperature_K=temperature, friction=0.1)
    md.attach(traj.write, interval=100)
    md.run(steps=5000)
    
    # Then run a longer NPT MD simulation to generate diverse training structures.
    md = NPT(
        atoms,
        1.0 * units.fs,
        temperature_K=temperature,
        externalstress=1.0 * units.bar,
        mask=np.eye(3),
        ttime=20.0 * units.fs,
        pfactor=2e6 * units.GPa * (units.fs**2)
    )
    md.attach(traj.write, interval=100)
    md.run(steps=steps_npt)

    md = NVTBerendsen(
        atoms,
        1.0 * units.fs,
        temperature_K=temperature,
        taut=500.0 * units.fs
    )
    rnemd = RNEMDExtension(
        md,
        rnemd_type="viscosity",
        n_slab=n_slab,
        n_swap=n_swap,
        swap_interval=swap_interval,
        log_interval=log_interval,
        logfile=logfile
    )
    log_extension = VelocityLogExtension(
        md,
        n_slab=n_slab,
        freq=10,
        interval=log_interval,
        logfile=profile
    )
    md.attach(traj.write, interval=100)
    md.attach(log_extension)
    md.attach(rnemd)
    md.run(steps=steps_rnemd)


In [ ]:
from ase.io import read
from pfp_api_client import ASECalculator, Estimator
from pathlib import Path

md_dir = Path("pfp_md_2")
md_dir.mkdir(exist_ok=True)

steps_npt = 20000
steps_rnemd = 50000
n_swap = 1
temperatures = [400, 400, 450, 450, 500, 500]
swap_intervals = [100, 500, 100, 500, 100, 500]
log_interval = 100

def md_warp(temperature, swap_interval):
    atoms = read("assets/test.xyz")
    calc = ASECalculator(Estimator(model_version=model_version, calc_mode=calc_mode))
    atoms.calc = calc
    name = f"{temperature}K_{swap_interval}"
    logfile = md_dir / f"{name}.log"
    profile = md_dir / f"profile_{name}.log"
    trajfile = md_dir / f"md_{name}.traj"
    md_script(atoms, temperature, steps_npt, steps_rnemd, n_swap, swap_interval, log_interval, logfile, profile, trajfile)


In [ ]:
from joblib.parallel import Parallel, delayed

Parallel(n_jobs=6)(delayed(md_warp)(temperature, swap_interval) for temperature, swap_interval in zip(temperatures, swap_intervals))

## 6. LightPFP validation MD

The rNEMD simulations are performed with LightPFP model using the exactly same conditions as described above.

In [ ]:
from light_pfp_client import ASECalculator, Estimator

calc = ASECalculator(Estimator(model_id = model_id))

In [ ]:
import numpy as np
from ase import units
from ase.io import Trajectory
from ase.md.langevin import Langevin
from ase.md.nvtberendsen import NVTBerendsen
from ase.md.npt import NPT
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from IPython.display import clear_output
from light_pfp_autogen.context import DataCollectionContext
from rnemd_extensions import RNEMDExtension, VelocityLogExtension


def md_script(atoms, temperature, steps_npt, steps_rnemd, n_swap, swap_interval, log_interval, logfile, profile, trajfile):
    n_slab = 20
    traj = Trajectory(trajfile, "w", atoms=atoms)
    MaxwellBoltzmannDistribution(atoms, temperature_K=temperature)
    
    # Run a short NVT MD simulation (e.g., using Langevin dynamics) to equilibrate the structure.
    md = Langevin(atoms, 1.0 * units.fs, temperature_K=temperature, friction=0.1)
    md.attach(traj.write, interval=100)
    md.run(steps=5000)
    
    # Then run a longer NPT MD simulation to generate diverse training structures.
    md = NPT(
        atoms,
        1.0 * units.fs,
        temperature_K=temperature,
        externalstress=1.0 * units.bar,
        mask=np.eye(3),
        ttime=20.0 * units.fs,
        pfactor=2e6 * units.GPa * (units.fs**2)
    )
    md.attach(traj.write, interval=100)
    md.run(steps=steps_npt)

    md = NVTBerendsen(
        atoms,
        1.0 * units.fs,
        temperature_K=temperature,
        taut=500.0 * units.fs
    )
    rnemd = RNEMDExtension(
        md,
        rnemd_type="viscosity",
        n_slab=n_slab,
        n_swap=n_swap,
        swap_interval=swap_interval,
        log_interval=log_interval,
        logfile=logfile
    )
    log_extension = VelocityLogExtension(
        md,
        n_slab=n_slab,
        freq=10,
        interval=log_interval,
        logfile=profile
    )
    md.attach(traj.write, interval=100)
    md.attach(log_extension)
    md.attach(rnemd)
    md.run(steps=steps_rnemd)
        

In [ ]:
steps_npt = 20000
steps_rnemd = 50000
n_swap = 1
temperatures = [400, 400, 450, 450, 500, 500]
swap_intervals = [100, 500, 100, 500, 100, 500]
log_interval = 100

In [ ]:
from ase.io import read
from pathlib import Path

md_dir = Path("light_pfp_md")
md_dir.mkdir(exist_ok=True)

for temperature, swap_interval in zip(temperatures, swap_intervals):
    atoms = read("assets/test.xyz")
    atoms.calc = calc
    name = f"{temperature}K_{swap_interval}"
    logfile = md_dir / f"{name}.log"
    profile = md_dir / f"profile_{name}.log"
    trajfile = md_dir / f"md_{name}.traj"
    md_script(atoms, temperature, steps_npt, steps_rnemd, n_swap, swap_interval, log_interval, logfile, profile, trajfile)

### 6.2 Comparison of MD trajectories

The MD trajectories of PFP-MD and LightPFP-MD are compared.

In [ ]:
import itertools

temperatures = [400, 450, 500]
swap_intervals = [100, 500]
params = list(itertools.product(temperatures, swap_intervals))

colors = ["r", "b", "g", "c", "m", "y"]
print(params)

#### a. Density

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ase import units
from ase.io import Trajectory

def get_density(atoms):
    return atoms.get_masses().sum() / units.kg / atoms.get_volume() * 1e27

In [ ]:
pfp_density = [
    [get_density(atoms) for atoms in Trajectory(f"pfp_md/md_{t}K_{interval}.traj")[:250]]
    for t, interval in params
]
lpfp_density = [
    [get_density(atoms) for atoms in Trajectory(f"light_pfp_md/md_{t}K_{interval}.traj")[:250]]
    for t, interval in params
]

* Plot the density variation of MD trajectries

<img src="assets/density.png" width=500>

In [ ]:
for (t, i), c, pfp, lpfp in zip(params, ["r", "r", "b", "b", "g", "g"], pfp_density, lpfp_density):
    plt.plot(np.arange(len(pfp))*0.1, pfp, label=f"PFP {t}K {i}", c=c)
    plt.plot(np.arange(len(lpfp))*0.1, lpfp, label=f"LightPFP {t}K {i}", c=c, linestyle="--")

plt.xlabel("time (ps)")
plt.ylabel("density (g/cm^3)")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.savefig("density.png")

* The density of the decane at 400, 450 and 500K are calculated from the last 10 ps MD trajectory from the NPT stage.

| Temperature | PFP | LightPFP |
|-|-|-|
|400K| 0.689| 0.675|
|450K| 0.647| 0.650|
|500K| 0.611| 0.617|

In [ ]:
print("Temperature PFP LightPFP")
for t in temperatures:
    pfp_ave = np.mean(
        [get_density(atoms) for atoms in Trajectory(f"pfp_md/md_{t}K_100.traj")[150:250]] + 
        [get_density(atoms) for atoms in Trajectory(f"pfp_md/md_{t}K_500.traj")[150:250]]
    )
    light_pfp_ave = np.mean(
        [get_density(atoms) for atoms in Trajectory(f"light_pfp_md/md_{t}K_100.traj")[150:250]] + 
        [get_density(atoms) for atoms in Trajectory(f"light_pfp_md/md_{t}K_500.traj")[150:250]]
    )
    print(f"{t}K {pfp_ave:.3f} {light_pfp_ave:.3f}")


### RDF

The radial distribution function of the decane at 400, 450 and 500K are calculated from the last 10 ps MD trajectory from the NPT stage.

<img src="assets/rdf_400K.png" width=500>

**RDF at 400K**

<img src="assets/rdf_450K.png" width=500>

**RDF at 450K**

<img src="assets/rdf_500K.png" width=500>

**RDF at 500K**


In [ ]:
from light_pfp_evaluate.md import plot_rdf
from ase.io import Trajectory

for t in temperatures:
    pfp_atoms_list = [atoms for atoms in Trajectory(f"pfp_md/md_{t}K_100.traj")[150:250:10]] + [atoms for atoms in Trajectory(f"pfp_md/md_{t}K_500.traj")[150:250:10]]
    light_pfp_atoms_list = [atoms for atoms in Trajectory(f"light_pfp_md/md_{t}K_100.traj")[150:250:10]] + [atoms for atoms in Trajectory(f"light_pfp_md/md_{t}K_500.traj")[150:250:10]]
    plot_rdf(
        [t],
        [pfp_atoms_list],
        [light_pfp_atoms_list],
        f"rdf_{t}K.png"
    )

### Viscosity

The viscosity of decane at 400, 450 and 500K are calulated and plotted against the experimental value from NIST database.

<img src="assets/viscosity.png" width=500>

In [ ]:
from ase import units
from rnemd_extensions import get_viscosity

pfp_viscosity = {}
lpfp_viscosity = {}
for (t, i) in params:
    pfp_viscosity[(t,i)] = get_viscosity(f"pfp_md/{t}K_{i}.log", f"pfp_md/profile_{t}K_{i}.log", units.fs, 20000)
    lpfp_viscosity[(t,i)] = get_viscosity(f"light_pfp_md/{t}K_{i}.log", f"light_pfp_md/profile_{t}K_{i}.log", units.fs, 20000)


In [ ]:
nist_temperatures = [400, 425, 446.75]
viscosities = [0.28922, 0.23823, 0.20320]

In [ ]:
plt.scatter(temperatures, [pfp_viscosity[(t, 100)] for t in temperatures], c="r", marker="o", label="PFP, interval=100")
plt.scatter(temperatures, [lpfp_viscosity[(t, 100)] for t in temperatures], c="b", marker="o", label="LightPFP, interval=100")
plt.scatter(temperatures, [pfp_viscosity[(t, 500)] for t in temperatures], c="r", marker="s", label="PFP, interval=500")
plt.scatter(temperatures, [lpfp_viscosity[(t, 500)] for t in temperatures], c="b", marker="s", label="LightPFP, interval=500")
plt.scatter(nist_temperatures, viscosities, c="g", marker="^", label="NIST")
plt.legend()
plt.xlabel("temperature (K)")
plt.ylabel("viscosity (mPa s)")
plt.legend()
plt.savefig("viscosity.png")

## 7. Large-scale MD

Large-scale and long MD simulation is performed with LigthPFP. The workflow is similar with what is described above, but some parameters are changed.

* MD workflow
    * Start from "asset/md_init.xyz" initial structure with 12288 atoms
    * Run NVT MD for 5 ps
    * Run NPT MD for 20 ps
    * Run NVT MD + rNEMD for 1000 ps
* Conditions for rNEMD
    * Number of slabs: 40
    * Exchange interval: 50 / 100 / 500
    * Calculation of velocity profile: every 10 MD step

In [ ]:
from light_pfp_client import ASECalculator, Estimator

calc = ASECalculator(Estimator(model_id = model_id))

In [ ]:
import numpy as np
from ase import units
from ase.io import Trajectory
from ase.md.langevin import Langevin
from ase.md.nvtberendsen import NVTBerendsen
from ase.md.npt import NPT
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from IPython.display import clear_output
from light_pfp_autogen.context import DataCollectionContext
from rnemd_extensions import RNEMDExtension, VelocityLogExtension


def md_script(atoms, temperature, steps_npt, steps_rnemd, n_swap, swap_interval, log_interval, logfile, profile, trajfile):
    n_slab = 40
    traj = Trajectory(trajfile, "w", atoms=atoms)
    MaxwellBoltzmannDistribution(atoms, temperature_K=temperature)
    
    # Run a short NVT MD simulation (e.g., using Langevin dynamics) to equilibrate the structure.
    md = Langevin(atoms, 1.0 * units.fs, temperature_K=temperature, friction=0.1)
    md.attach(traj.write, interval=1000)
    md.run(steps=5000)
    
    # Then run a longer NPT MD simulation to generate diverse training structures.
    md = NPT(
        atoms,
        1.0 * units.fs,
        temperature_K=temperature,
        externalstress=1.0 * units.bar,
        mask=np.eye(3),
        ttime=20.0 * units.fs,
        pfactor=2e6 * units.GPa * (units.fs**2)
    )
    md.attach(traj.write, interval=1000)
    md.run(steps=steps_npt)

    md = NVTBerendsen(
        atoms,
        1.0 * units.fs,
        temperature_K=temperature,
        taut=500.0 * units.fs
    )
    rnemd = RNEMDExtension(
        md,
        rnemd_type="viscosity",
        n_slab=n_slab,
        n_swap=n_swap,
        swap_interval=swap_interval,
        log_interval=log_interval,
        logfile=logfile
    )
    log_extension = VelocityLogExtension(
        md,
        n_slab=n_slab,
        freq=10,
        interval=log_interval,
        logfile=profile
    )
    md.attach(traj.write, interval=10000)
    md.attach(log_extension)
    md.attach(rnemd)
    md.run(steps=steps_rnemd)
        

In [ ]:
steps_npt = 20000
steps_rnemd = 1000000
n_swap = 1
temperatures = [300, 300, 300, 350, 350, 350, 400, 400, 450, 450, 500, 500]
swap_intervals = [50, 100, 500, 50, 100, 500, 100, 500, 100, 500, 100, 500]
log_interval = 100

In [ ]:
from ase.io import read
from pathlib import Path

md_dir = Path("md")
md_dir.mkdir(exist_ok=True)

for temperature, swap_interval in zip(temperatures, swap_intervals):
    atoms = read("md_init.xyz")
    atoms.calc = calc
    name = f"{temperature}K_{swap_interval}"
    logfile = md_dir / f"{name}.log"
    profile = md_dir / f"profile_{name}.log"
    trajfile = md_dir / f"md_{name}.traj"
    md_script(atoms, temperature, steps_npt, steps_rnemd, n_swap, swap_interval, log_interval, logfile, profile, trajfile)

### 7.1 Viscosity at low temperature

The viscosity of decane at 300, 350 and 400K are calulated and plotted against the experimental value from NIST database.

<img src="assets/viscosity_large.png" width=500>

In [ ]:
from ase import units
from rnemd_extensions import get_viscosity

pfp_viscosity = {}
lpfp_viscosity = {}
for (t, i) in params:
    pfp_viscosity[(t,i)] = get_viscosity(f"pfp_md/{t}K_{i}.log", f"pfp_md/profile_{t}K_{i}.log", units.fs, 20000)
    lpfp_viscosity[(t,i)] = get_viscosity(f"light_pfp_md/{t}K_{i}.log", f"light_pfp_md/profile_{t}K_{i}.log", units.fs, 20000)


In [ ]:
nist_temperatures = [400, 425, 446.75]
viscosities = [0.28922, 0.23823, 0.20320]

In [ ]:
plt.scatter([300, 350, 400], [viscosity[(t, 500)] for t in [300, 350, 400]], c="b", marker="s", label="interval=500")
plt.scatter([300, 350, 400], [viscosity[(t, 100)] for t in [300, 350, 400]], c="r", marker="o", label="interval=100")
plt.scatter([300, 350], [viscosity[(t, 50)] for t in [300, 350]], c="y", marker="s", label="interval=50")
plt.scatter([300, 350, 400], [0.82576, 0.45343, 0.28922], c="g", marker="^", label="NIST")
plt.legend()
plt.xlabel("temperature (K)")
plt.ylabel("viscosity (mPa s)")
plt.legend()
plt.savefig("viscosity_large.png")